In [1]:
# Import libraries
import pandas as pd
import datetime as dt
import numpy as np
import re 



In [2]:
df = pd.read_csv('online_retail_II.csv')
df

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,12/1/2009 7:45,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,12/1/2009 7:45,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,12/1/2009 7:45,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,12/1/2009 7:45,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,12/1/2009 7:45,1.25,13085.0,United Kingdom
...,...,...,...,...,...,...,...,...
525456,538171,22271,FELTCRAFT DOLL ROSIE,2,12/9/2010 20:01,2.95,17530.0,United Kingdom
525457,538171,22750,FELTCRAFT PRINCESS LOLA DOLL,1,12/9/2010 20:01,3.75,17530.0,United Kingdom
525458,538171,22751,FELTCRAFT PRINCESS OLIVIA DOLL,1,12/9/2010 20:01,3.75,17530.0,United Kingdom
525459,538171,20970,PINK FLORAL FELTCRAFT SHOULDER BAG,2,12/9/2010 20:01,3.75,17530.0,United Kingdom


- Invoice: số hóa đơn: một số nguyên gồm 6 chữ số được gán duy nhất cho mỗi giao dịch
- Stockcode: mã sản phẩm: một số nguyên gồm 5 chữ số được gán cho từng sản phẩm riêng biệt
- Description: mô tả sản phẩm
- Quantity: số lượng từng sản phẩm trên mỗi giao dịch
- InvoiceDate: ngày và giờ lập hóa đơn
- Price: đơn giá: giá sản phẩm trên mỗi đơn vị
- CustomerID: mã số khách hàng: số nguyên gồm 5 chữ số được gán duy nhất cho mỗi khách hàng
- Country: quốc gia mà khách hàng cư trú

In [3]:
df.nunique()

Invoice        28816
StockCode       4632
Description     4681
Quantity         825
InvoiceDate    25296
Price           1606
Customer ID     4383
Country           40
dtype: int64

In [4]:
df.describe()

,Quantity,Price,Customer ID
count,525461.000000,525461.000000,417534.000000
mean,10.337667,4.688834,15360.645478
std,107.424110,146.126914,1680.811316
min,-9600.000000,-53594.360000,12346.000000
25%,1.000000,1.250000,13983.000000
50%,3.000000,2.100000,15311.000000
75%,10.000000,4.210000,16799.000000
max,19152.000000,25111.090000,18287.000000


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 525461 entries, 0 to 525460
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   Invoice      525461 non-null  object 
 1   StockCode    525461 non-null  object 
 2   Description  522533 non-null  object 
 3   Quantity     525461 non-null  int64  
 4   InvoiceDate  525461 non-null  object 
 5   Price        525461 non-null  float64
 6   Customer ID  417534 non-null  float64
 7   Country      525461 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 32.1+ MB


Những phát hiện ban đầu:
- Dấu hiệu tiềm năng của các đơn đặt hàng bị hủy từ đơn giá âm: Việc có đơn giá âm không là không phổ biến vì điều này có nghĩa là một dòng tiền chảy vào công ty. Các giao dịch này có thể dại diện cho các đơn đặt hàng bị khách hủy hoặc nợ xấu/ xóa nợ mà doanh nghiệp phải gánh chịu
- Thiếu một lượng ID khách hàng

**TIỀN XỬ LÝ DỮ LIỆU**



**InvoiceDate**

Tách thông tin ngày và giờ ra khỏi InvoiceDate
Cột InvoiceDate chứa cả ngày và giờ của giao dịch. Những dữ liệu này được tách thành các cột riêng rẻ để tạo điều kiện thuận lợi cho việc xử lý dữ liệu và tính toán trong tương lai

In [6]:
#Chuyển đổi kiểu dữ liệu của InvoiceDate thành dạng datetime
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])
# Tách ngày giờ khỏi InvoiceDate
df['Date'] = df['InvoiceDate'].dt.date
df['Time'] = df['InvoiceDate'].dt.time

# Xóa cột InvoiceDate
df.drop(['InvoiceDate'], axis=1, inplace=True)

df.head()

,Invoice,StockCode,Description,Quantity,Price,Customer ID,Country,Date,Time
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,6.95,13085.0,United Kingdom,2009-12-01,07:45:00
1,489434,79323P,PINK CHERRY LIGHTS,12,6.75,13085.0,United Kingdom,2009-12-01,07:45:00
2,489434,79323W,WHITE CHERRY LIGHTS,12,6.75,13085.0,United Kingdom,2009-12-01,07:45:00
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2.10,13085.0,United Kingdom,2009-12-01,07:45:00
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,1.25,13085.0,United Kingdom,2009-12-01,07:45:00


**InvoiceNo**

Trích xuất trạng thái giao dịch từ cột Invoice
Invoice chứa cả trạng thái giao dịch (tức là có chữ 'C' biểu thị giao dịch bị hủy) và số nhận dạng hóa đơn (ví dụ: số hóa đơn duy nhất)

In [7]:
# Tách biệt trạng thái đơn hàng và số hóa đơn khỏi InvoiceNo
df['CancelledOrder'] = df['Invoice'].apply(
    lambda x: re.findall(r'[A-Z]', str(x))).apply(lambda x: pd.Series(x))
df['Invoice_No'] = df['Invoice'].apply(
    lambda x: re.findall(r'\d+', str(x))).apply(lambda x: pd.Series(x))

# loại bỏ cột Invoice cũ
df.drop(['Invoice'], axis=1, inplace=True)

df.head()

,StockCode,Description,Quantity,Price,Customer ID,Country,Date,Time,CancelledOrder,Invoice_No
0,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,6.95,13085.0,United Kingdom,2009-12-01,07:45:00,NaN,489434
1,79323P,PINK CHERRY LIGHTS,12,6.75,13085.0,United Kingdom,2009-12-01,07:45:00,NaN,489434
2,79323W,WHITE CHERRY LIGHTS,12,6.75,13085.0,United Kingdom,2009-12-01,07:45:00,NaN,489434
3,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2.10,13085.0,United Kingdom,2009-12-01,07:45:00,NaN,489434
4,21232,STRAWBERRY CERAMIC TRINKET BOX,24,1.25,13085.0,United Kingdom,2009-12-01,07:45:00,NaN,489434


In [8]:
# Mã hóa đơn hàng đã hủy
df['CancelledOrder'] = df['CancelledOrder'].astype('category')
df['CancelledOrder'].unique()

[NaN, 'C', 'A']
Categories (2, object): ['A', 'C']

In [9]:
def filter_row(df, column, criterion, operator='equal'):
    '''
    Filter rows based on specific condition
    '''
    if operator == 'equal':
        return df[df[column] == criterion]
    if operator == 'less':
        return df[df[column] <= criterion]
    if operator == 'more':
        return df[df[column] >= criterion]

def remove_row(df, column, criterion):
    '''
    Remove ros based on specific condition
    '''
    return df[df[column] != criterion]

In [10]:
# Hạng mục 'A' không ngờ tới ; in ra các hàng để điều tra thêm
filter_row(df, 'CancelledOrder', 'A')

,StockCode,Description,Quantity,Price,Customer ID,Country,Date,Time,CancelledOrder,Invoice_No
179403,B,Adjust bad debt,1,-53594.36,NaN,United Kingdom,2010-04-29,13:36:00,A,506401
276274,B,Adjust bad debt,1,-44031.79,NaN,United Kingdom,2010-07-19,11:24:00,A,516228
403472,B,Adjust bad debt,1,-38925.87,NaN,United Kingdom,2010-10-20,12:04:00,A,528059


Bỏ hồ sơ nợ xấu
- Các khoản điều chỉnh nợ xấu bị loại khỏi tập dữ liệu vì chúng không thể hiện doanh thu thực tế. Hơn nữa, chúng không được gắn thẻ cho bất kỳ khách hàng cụ thể nào.

In [11]:
# Xóa hồ sơ nợ xấu
df = remove_row(df, 'CancelledOrder', 'A')

# Mã hóa cột
df['CancelledOrder'] = df['CancelledOrder'].cat.add_categories([0])
df['CancelledOrder'].fillna(value=0, inplace=True)
df['CancelledOrder'].replace(to_replace='C', value=1, inplace=True)

df.head()

C:\Users\Adin\AppData\Local\Temp\ipykernel_22804\272757461.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['CancelledOrder'] = df['CancelledOrder'].cat.add_categories([0])
C:\Users\Adin\AppData\Local\Temp\ipykernel_22804\272757461.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['CancelledOrder'].fillna(value=0, inplace=True)
C:\Users\Adin\AppData\Local\Temp\ipykernel_22804\272757461.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-

,StockCode,Description,Quantity,Price,Customer ID,Country,Date,Time,CancelledOrder,Invoice_No
0,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,6.95,13085.0,United Kingdom,2009-12-01,07:45:00,0,489434
1,79323P,PINK CHERRY LIGHTS,12,6.75,13085.0,United Kingdom,2009-12-01,07:45:00,0,489434
2,79323W,WHITE CHERRY LIGHTS,12,6.75,13085.0,United Kingdom,2009-12-01,07:45:00,0,489434
3,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2.10,13085.0,United Kingdom,2009-12-01,07:45:00,0,489434
4,21232,STRAWBERRY CERAMIC TRINKET BOX,24,1.25,13085.0,United Kingdom,2009-12-01,07:45:00,0,489434


**StockCode dưới dạng dữ liệu phân loại**
- StockCode là mã định đanh duy nhất được gán cho từng mặt hàng và StockCode phải là một loại dtype

In [12]:
df['StockCode'] = df.StockCode.astype('category')

df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 525458 entries, 0 to 525460
Data columns (total 10 columns):
 #   Column          Non-Null Count   Dtype   
---  ------          --------------   -----   
 0   StockCode       525458 non-null  category
 1   Description     522530 non-null  object  
 2   Quantity        525458 non-null  int64   
 3   Price           525458 non-null  float64 
 4   Customer ID     417534 non-null  float64 
 5   Country         525458 non-null  object  
 6   Date            525458 non-null  object  
 7   Time            525458 non-null  object  
 8   CancelledOrder  525458 non-null  category
 9   Invoice_No      525458 non-null  object  
dtypes: category(2), float64(2), int64(1), object(5)
memory usage: 37.7+ MB


C:\Users\Adin\AppData\Local\Temp\ipykernel_22804\3055809550.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['StockCode'] = df.StockCode.astype('category')


**Đơn giá**

Đơn giá thể hiện giá trị của từng mặt hàng; một cột mới TotalSum có thể được tạo để thể hiện tổng giá mà khách hàng phải trả trên một giao dịch tương ứng

In [13]:
df['TotalSum'] = df['Quantity'] * df['Price']
df.describe()

C:\Users\Adin\AppData\Local\Temp\ipykernel_22804\1332332889.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['TotalSum'] = df['Quantity'] * df['Price']


,Quantity,Price,Customer ID,TotalSum
count,525458.000000,525458.000000,417534.000000,525458.000000
mean,10.337721,4.948734,15360.645478,18.414482
std,107.424415,96.493161,1680.811316,116.865637
min,-9600.000000,0.000000,12346.000000,-25111.090000
25%,1.000000,1.250000,13983.000000,3.750000
50%,3.000000,2.100000,15311.000000,9.950000
75%,10.000000,4.210000,16799.000000,17.700000
max,19152.000000,25111.090000,18287.000000,25111.090000


In [14]:
# Xem các hàng có Tổng số 0
filter_row(df, 'TotalSum', 0).head()


,StockCode,Description,Quantity,Price,Customer ID,Country,Date,Time,CancelledOrder,Invoice_No,TotalSum
263,21733,85123a mixed,-96,0.0,NaN,United Kingdom,2009-12-01,10:52:00,0,489464,-0.0
283,71477,short,-240,0.0,NaN,United Kingdom,2009-12-01,10:52:00,0,489463,-0.0
284,85123A,21733 mixed,-192,0.0,NaN,United Kingdom,2009-12-01,10:53:00,0,489467,-0.0
470,21646,NaN,-50,0.0,NaN,United Kingdom,2009-12-01,11:44:00,0,489521,-0.0
3114,20683,NaN,-44,0.0,NaN,United Kingdom,2009-12-01,17:26:00,0,489655,-0.0


Các hàng có 0 TotalSum dường như đóng vai trò ghi lại các hoạt động linh tinh; Cần thảo luận thêm với các nhà phân tích kinh doanh để hiểu bản chất của dữ liệu đó. Trong khi chờ xử lý như vậy, các hàng này sẽ bị xóa.

In [15]:
df = remove_row(df, 'TotalSum', 0)
df.describe()

,Quantity,Price,Customer ID,TotalSum
count,521771.000000,521771.000000,417503.000000,521771.000000
mean,10.768153,4.983703,15360.731300,18.544604
std,90.821824,96.832587,1680.779044,117.267527
min,-9360.000000,0.001000,12346.000000,-25111.090000
25%,1.000000,1.250000,13983.000000,3.750000
50%,3.000000,2.100000,15311.000000,10.000000
75%,10.000000,4.210000,16799.000000,17.700000
max,19152.000000,25111.090000,18287.000000,25111.090000


In [16]:
# View rows with 0 TotalSum
filter_row(df, 'TotalSum', 0, 'less')

,StockCode,Description,Quantity,Price,Customer ID,Country,Date,Time,CancelledOrder,Invoice_No,TotalSum
178,22087,PAPER BUNTING WHITE LACE,-12,2.95,16321.0,Australia,2009-12-01,10:33:00,1,489449,-35.40
179,85206A,CREAM FELT EASTER EGG BASKET,-6,1.65,16321.0,Australia,2009-12-01,10:33:00,1,489449,-9.90
180,21895,POTTING SHED SOW 'N' GROW SET,-4,4.25,16321.0,Australia,2009-12-01,10:33:00,1,489449,-17.00
181,21896,POTTING SHED TWINE,-6,2.10,16321.0,Australia,2009-12-01,10:33:00,1,489449,-12.60
182,22083,PAPER CHAIN KIT RETRO SPOT,-12,2.95,16321.0,Australia,2009-12-01,10:33:00,1,489449,-35.40
...,...,...,...,...,...,...,...,...,...,...,...
524695,22956,36 FOIL HEART CAKE CASES,-2,2.10,12605.0,Germany,2010-12-09,15:41:00,1,538123,-4.20
524696,M,Manual,-4,0.50,15329.0,United Kingdom,2010-12-09,15:43:00,1,538124,-2.00
524697,22699,ROSES REGENCY TEACUP AND SAUCER,-1,2.95,15329.0,United Kingdom,2010-12-09,15:43:00,1,538124,-2.95
524698,22423,REGENCY CAKESTAND 3 TIER,-1,12.75,15329.0,United Kingdom,2010-12-09,15:43:00,1,538124,-12.75


In [17]:
# Kiểm tra xem có Tổng nào nhỏ hơn 0 không thuộc lệnh bị hủy
df[df['TotalSum'] <= 0][df['CancelledOrder'] == 0].head()


C:\Users\Adin\AppData\Local\Temp\ipykernel_22804\1474627464.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df[df['TotalSum'] <= 0][df['CancelledOrder'] == 0].head()


,StockCode,Description,Quantity,Price,Customer ID,Country,Date,Time,CancelledOrder,Invoice_No,TotalSum


**Description**
- Mô tả có thể chứa dữ liệu không liên quan
- Mục không liên quan như vậy sẽ bị xóa 

In [18]:
print(df['Description'].unique())

print('\n Number of unique items: {}'.format(df['Description'].nunique()))

['15CM CHRISTMAS GLASS BALL 20 LIGHTS' 'PINK CHERRY LIGHTS'
 ' WHITE CHERRY LIGHTS' ... 'BAKING MOULD CHOCOLATE CUP CAKES'
 'BAKING MOULD EASTER EGG MILK CHOC' '*Boombox Ipod Classic']

 Number of unique items: 4549


**Quantity**

In [19]:
# Check for rows with negative quantity
filter_row(df, 'Quantity', 0, 'less')

,StockCode,Description,Quantity,Price,Customer ID,Country,Date,Time,CancelledOrder,Invoice_No,TotalSum
178,22087,PAPER BUNTING WHITE LACE,-12,2.95,16321.0,Australia,2009-12-01,10:33:00,1,489449,-35.40
179,85206A,CREAM FELT EASTER EGG BASKET,-6,1.65,16321.0,Australia,2009-12-01,10:33:00,1,489449,-9.90
180,21895,POTTING SHED SOW 'N' GROW SET,-4,4.25,16321.0,Australia,2009-12-01,10:33:00,1,489449,-17.00
181,21896,POTTING SHED TWINE,-6,2.10,16321.0,Australia,2009-12-01,10:33:00,1,489449,-12.60
182,22083,PAPER CHAIN KIT RETRO SPOT,-12,2.95,16321.0,Australia,2009-12-01,10:33:00,1,489449,-35.40
...,...,...,...,...,...,...,...,...,...,...,...
524695,22956,36 FOIL HEART CAKE CASES,-2,2.10,12605.0,Germany,2010-12-09,15:41:00,1,538123,-4.20
524696,M,Manual,-4,0.50,15329.0,United Kingdom,2010-12-09,15:43:00,1,538124,-2.00
524697,22699,ROSES REGENCY TEACUP AND SAUCER,-1,2.95,15329.0,United Kingdom,2010-12-09,15:43:00,1,538124,-2.95
524698,22423,REGENCY CAKESTAND 3 TIER,-1,12.75,15329.0,United Kingdom,2010-12-09,15:43:00,1,538124,-12.75


**Số lượng âm biểu thị cho giao dịch bị hủy**
- Có vẻ như số lượng âm thể hiện giao dịch đã bị hủy. Tuy nhiên, lý do chính xác cho việc hủy bỏ vẫn chưa được biết.
- Có hai cách để quản lý các đơn hàng bị hủy:
 Tính đến cả đơn hàng ban đầu tương ứng và đơn hàng bị hủy để xóa doanh số bán hàng khỏi tập dữ liệu vì không có lợi nhuận thực tế nào được tạo ra từ các đơn hàng bị hủy
 Chỉ bỏ các đơn hàng đã hủy để tối đa hóa dữ liệu của khách hàng, tuy nhiên nó cũng sẽ ghi lại việc khách hàng mua sai.
 
(THẢO LUẬN) Cách tiếp cận đầu tiên sẽ phản ánh giao dịch mua thực tế đã thực hiện, vì các đơn đặt hàng bị hủy có thể cho thấy khách hàng đã đặt hàng sai và điều này không phản ánh ý định mua hàng thực tế của khách hàng. Giá trị của các giao dịch mua tương ứng và giao dịch bị hủy sẽ được tổng hợp và do đó giá trị của các hàng sẽ bù trừ cho nhau một cách tự nhiên.

**CustomerID**

In [20]:
df.describe()

,Quantity,Price,Customer ID,TotalSum
count,521771.000000,521771.000000,417503.000000,521771.000000
mean,10.768153,4.983703,15360.731300,18.544604
std,90.821824,96.832587,1680.779044,117.267527
min,-9360.000000,0.001000,12346.000000,-25111.090000
25%,1.000000,1.250000,13983.000000,3.750000
50%,3.000000,2.100000,15311.000000,10.000000
75%,10.000000,4.210000,16799.000000,17.700000
max,19152.000000,25111.090000,18287.000000,25111.090000


In [21]:
# Xác định các hàng bị thiếu ID khách hàng
df.isnull().sum(axis=0)

StockCode              0
Description            0
Quantity               0
Price                  0
Customer ID       104268
Country                0
Date                   0
Time                   0
CancelledOrder         0
Invoice_No             0
TotalSum               0
dtype: int64

20% thông tin quan trọng bị thiếu trong tập dữ liệu
- Có ~20% tổng dữ liệu bị thiếu thông tin về CustomerID. CustomerID chứa danh tính của khách hàng và nếu không có ID đó thì sẽ không thể thực hiện phân khúc khách hàng.

Khám phá việc tính toán dữ liệu dựa trên hóa đơnKhông
- Các giá trị còn thiếu có thể được tính toán dựa trên các tính năng khác như InvoiceNo vì cùng một khách hàng có thể sẽ mua các mặt hàng theo cùng một hóa đơn.

In [22]:
# Print rows with missing CustomerID
df[df['Customer ID'].isnull()]

,StockCode,Description,Quantity,Price,Customer ID,Country,Date,Time,CancelledOrder,Invoice_No,TotalSum
577,85226C,BLUE PULL BACK RACING CAR,1,0.55,NaN,United Kingdom,2009-12-01,11:49:00,0,489525,0.55
578,85227,SET/6 3D KIT CARDS FOR KIDS,1,0.85,NaN,United Kingdom,2009-12-01,11:49:00,0,489525,0.85
1055,22271,FELTCRAFT DOLL ROSIE,1,2.95,NaN,United Kingdom,2009-12-01,12:32:00,0,489548,2.95
1056,22254,FELT TOADSTOOL LARGE,12,1.25,NaN,United Kingdom,2009-12-01,12:32:00,0,489548,15.00
1057,22273,FELTCRAFT DOLL MOLLY,3,2.95,NaN,United Kingdom,2009-12-01,12:32:00,0,489548,8.85
...,...,...,...,...,...,...,...,...,...,...,...
525143,82599,FANNY'S REST STOPMETAL SIGN,1,4.21,NaN,United Kingdom,2010-12-09,16:35:00,0,538154,4.21
525144,84029E,RED WOOLLY HOTTIE WHITE HEART.,5,8.47,NaN,United Kingdom,2010-12-09,16:35:00,0,538154,42.35
525145,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,4,8.47,NaN,United Kingdom,2010-12-09,16:35:00,0,538154,33.88
525146,85099B,JUMBO BAG RED RETROSPOT,1,4.21,NaN,United Kingdom,2010-12-09,16:35:00,0,538154,4.21


In [23]:
df 

,StockCode,Description,Quantity,Price,Customer ID,Country,Date,Time,CancelledOrder,Invoice_No,TotalSum
0,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,6.95,13085.0,United Kingdom,2009-12-01,07:45:00,0,489434,83.40
1,79323P,PINK CHERRY LIGHTS,12,6.75,13085.0,United Kingdom,2009-12-01,07:45:00,0,489434,81.00
2,79323W,WHITE CHERRY LIGHTS,12,6.75,13085.0,United Kingdom,2009-12-01,07:45:00,0,489434,81.00
3,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2.10,13085.0,United Kingdom,2009-12-01,07:45:00,0,489434,100.80
4,21232,STRAWBERRY CERAMIC TRINKET BOX,24,1.25,13085.0,United Kingdom,2009-12-01,07:45:00,0,489434,30.00
...,...,...,...,...,...,...,...,...,...,...,...
525456,22271,FELTCRAFT DOLL ROSIE,2,2.95,17530.0,United Kingdom,2010-12-09,20:01:00,0,538171,5.90
525457,22750,FELTCRAFT PRINCESS LOLA DOLL,1,3.75,17530.0,United Kingdom,2010-12-09,20:01:00,0,538171,3.75
525458,22751,FELTCRAFT PRINCESS OLIVIA DOLL,1,3.75,17530.0,United Kingdom,2010-12-09,20:01:00,0,538171,3.75
525459,20970,PINK FLORAL FELTCRAFT SHOULDER BAG,2,3.75,17530.0,United Kingdom,2010-12-09,20:01:00,0,538171,7.50


**TÍNH TOÁN RFM**

In [24]:
# Tính phạm vi ngày 1 năm từ dữ liệu mới nhất
earliest_date = df['Date'].min()
end_date = df['Date'].max()

print("Actual Start Date: {}, Actual End Date: {}".format(earliest_date, end_date))

# Lọc phạm vi dữ liệu 1 năm từ df gốc
start_date = end_date - pd.to_timedelta(364, unit='d')
df_rfm = df[(df['Date'] >= start_date) & (df['Date'] <= end_date)]

print("RFM Start Date: {}, RFM End Date: {}".format(
    df_rfm['Date'].min(), df_rfm['Date'].max()))

Actual Start Date: 2009-12-01, Actual End Date: 2010-12-09
RFM Start Date: 2009-12-10, RFM End Date: 2010-12-09


In [25]:
#Tạo snapshotdate giả định
snapshot_date = end_date + dt.timedelta(days=1)

# Tính giá trị gần đây, tần suất và tiền tệ cho mỗi khách hàng
df_rfm = df.groupby(['Customer ID']).agg({
    'Date': lambda x: (snapshot_date - x.max()).days,
    'Invoice_No': 'count',
    'TotalSum': 'sum'})

# Rename the columns
df_rfm.rename(columns={'Date': 'Recency',
                       'Invoice_No': 'Frequency',
                       'TotalSum': 'MonetaryValue'}, inplace=True)

# Print top 5 rows
print(df_rfm.head())

             Recency  Frequency  MonetaryValue
Customer ID                                   
12346.0           67         46         -64.68
12347.0            3         71        1323.32
12348.0           74         20         222.16
12349.0           43        107        2646.99
12351.0           11         21         300.93


In [26]:
df_rfm

,Recency,Frequency,MonetaryValue
Customer ID,,,
12346.0,67,46,-64.68
12347.0,3,71,1323.32
12348.0,74,20,222.16
12349.0,43,107,2646.99
12351.0,11,21,300.93
...,...,...,...
18283.0,18,230,641.77
18284.0,65,29,436.68
18285.0,296,12,427.00


In [27]:
# Lọc các dòng có giá trị âm trong cột 'R', 'F', hoặc 'M'
rows_with_negative_values = df_rfm[(df_rfm['Recency'] < 0) | (df_rfm['Frequency'] < 0) | (df_rfm['MonetaryValue'] < 0)]

# In ra các dòng chứa giá trị âm
print(rows_with_negative_values)

             Recency  Frequency  MonetaryValue
Customer ID                                   
12346.0           67         46         -64.68
12382.0          318          1         -18.38
12590.0          137          2          -7.90
12706.0          318          1         -91.89
12896.0          366          1         -29.75
...              ...        ...            ...
17645.0          364          1          -9.90
17661.0          277          1         -54.00
17755.0           60         10         -72.23
17943.0          315          1        -165.03
18023.0          176          1       -3248.86

[90 rows x 3 columns]


In [28]:
df_rfm = df_rfm.drop(rows_with_negative_values.index)
df_rfm

,Recency,Frequency,MonetaryValue
Customer ID,,,
12347.0,3,71,1323.32
12348.0,74,20,222.16
12349.0,43,107,2646.99
12351.0,11,21,300.93
12352.0,11,18,343.80
...,...,...,...
18283.0,18,230,641.77
18284.0,65,29,436.68
18285.0,296,12,427.00


In [29]:
# In ra giá trị lớn nhất và nhỏ nhất của từng cột 'R', 'F', 'M'
print("Max values:")
print("MaxR:", df_rfm['Recency'].max())
print("MaxF:", df_rfm['Frequency'].max())
print("MaxM:", df_rfm['MonetaryValue'].max())

print("\nMin values:")
print("MinR:", df_rfm['Recency'].min())
print("MinF:", df_rfm['Frequency'].min())
print("MinM:", df_rfm['MonetaryValue'].min())

Max values:
MaxR: 374
MaxF: 5710
MaxM: 341776.73

Min values:
MinR: 1
MinF: 1
MinM: 0.0


**PHÂN CỤM FUZZY C-MEANS (FCM)**

**Tính chỉ số Fuzzy PBM index để tìm ra số cụm tối ưu nhất**

1. Import các thư viện cần thiết:

In [30]:
import pandas as pd
import numpy as np

2. Định nghĩa hàm tính Fuzzy PBM index:

In [31]:
def calculate_pbm_index(data, centers, u_matrix):
    n_clusters = len(centers)
    n_samples = data.shape[0]
    pbm_index = 0

    for i in range(n_samples):
        max_membership = max(u_matrix[i])
        min_distance = np.inf

        for j in range(n_clusters):
            distance = np.linalg.norm(data[i] - centers[j])
            if distance < min_distance:
                min_distance = distance

        pbm_index += max_membership * min_distance

    return pbm_index

3. Định nghĩa hàm thực hiện Fuzzy C-Means:

In [32]:
def fuzzy_cmeans(data, n_clusters, m, max_iter):
    n_samples = data.shape[0]
    n_features = data.shape[1]

    # Khởi tạo ma trận U ban đầu với giá trị ngẫu nhiên
    u_matrix = np.random.rand(n_samples, n_clusters)
    u_matrix = u_matrix / np.sum(u_matrix, axis=1, keepdims=True)

    # Khởi tạo ma trận centers ban đầu
    centers = np.random.rand(n_clusters, n_features)

    for _ in range(max_iter):
        # Cập nhật centers
        for j in range(n_clusters):
            numerator = np.sum((u_matrix[:, j] ** m).reshape(-1, 1) * data, axis=0)
            denominator = np.sum(u_matrix[:, j] ** m)
            centers[j] = numerator / denominator

        # Cập nhật ma trận U
        distances = np.linalg.norm(data[:, np.newaxis] - centers, axis=2)
        u_matrix = 1 / (distances ** (2 / (m - 1)))
        u_matrix = u_matrix / np.sum(u_matrix, axis=1, keepdims=True)

    return centers, u_matrix

4. Xác định số lượng cụm tối ưu:

In [33]:
best_pbm_index = -np.inf
best_n_clusters = 0

for n_clusters in range(2, 11): # Thử từ 2 đến 10 cụm
    centers, u_matrix = fuzzy_cmeans(df_rfm.values, n_clusters, m=2, max_iter=100)
    pbm_index = calculate_pbm_index(df_rfm.values, centers, u_matrix)

    if pbm_index > best_pbm_index:
        best_pbm_index = pbm_index
        best_n_clusters = n_clusters

print("Số lượng cụm tối ưu: ", best_n_clusters)

Số lượng cụm tối ưu:  2


In [34]:
best_pbm_index

7485610.698247655

Trong mã trên, chúng ta thử từ 2 đến 10 cụm và tính giá trị Fuzzy PBM index tương ứng. Số lượng cụm tối ưu sẽ là giá trị tương ứng với Fuzzy PBM index lớn nhất.



**Tiếp theo là áp dụng Fuzzy C-Means với số lượng cụm tối ưu là 2**

4. Áp dụng Fuzzy C-Means với số lượng cụm tối ưu:

In [35]:
n_clusters = best_n_clusters  # Số lượng cụm tối ưu từ bước trước
m = 2  # Hệ số m trong Fuzzy C-Means
max_iter = 100  # Số lần lặp tối đa

centers, u_matrix = fuzzy_cmeans(df_rfm.values, n_clusters, m, max_iter)

# In ra trung tâm cụm
print(centers)



# Tạo một cột mới trong dataframe để lưu nhãn cụm
df_rfm['Cluster'] = np.argmax(u_matrix, axis=1)

# Hiển thị kết quả
print(df_rfm.head())

[[5.91868851e+00 2.24706331e+03 1.97723671e+05]
 [8.87897229e+01 9.37209882e+01 1.67694490e+03]]
             Recency  Frequency  MonetaryValue  Cluster
Customer ID                                            
12347.0            3         71        1323.32        1
12348.0           74         20         222.16        1
12349.0           43        107        2646.99        1
12351.0           11         21         300.93        1
12352.0           11         18         343.80        1


In [36]:
# In ra ma trận trọng số cụm
print(u_matrix)

[[3.44566359e-06 9.99996554e-01]
 [5.43921737e-05 9.99945608e-01]
 [2.47832294e-05 9.99975217e-01]
 ...
 [4.14042182e-05 9.99958596e-01]
 [6.20609474e-06 9.99993794e-01]
 [1.16690981e-05 9.99988331e-01]]


Trong mã trên, chúng ta áp dụng Fuzzy C-Means với số lượng cụm tối ưu đã xác định từ bước trước. Kết quả sẽ được lưu trong cột "Cluster" của DataFrame df_rfm.



**ÁP DỤNG FUZZY AHP**

In [37]:
# Hàm tính ma trận trọng số Fuzzy AHP từ ma trận so sánh đôi
def fuzzy_ahp(matrix):
    n = matrix.shape[0]
    # Chuẩn hóa ma trận so sánh đôi
    normalized_matrix = matrix / matrix.sum(axis=0)
    # Tính trung bình cộng của các giá trị chuẩn hóa
    average_normalized = normalized_matrix.mean(axis=1)
    # Chuẩn hóa trung bình cộng
    normalized_average_normalized = average_normalized / average_normalized.sum()
    return normalized_average_normalized

# Ma trận so sánh đôi cho Recency (R), Frequency (F), Monetary Value (M)
comparison_matrix = np.array([
    [1, 2, 3],  # Ví dụ: R so với F (R=1, F=2, M=3)
    [0.5, 1, 2],  # F so với M
    [1/3, 0.5, 1]  # M so với R
])

# Tính ma trận trọng số Fuzzy AHP
weights_matrix = fuzzy_ahp(comparison_matrix)

# Trọng số cuối cùng cho R, F, M
R_weight = weights_matrix[0]
F_weight = weights_matrix[1]
M_weight = weights_matrix[2]

print("Trọng số cuối cùng cho Recency (R):", R_weight)
print("Trọng số cuối cùng cho Frequency (F):", F_weight)
print("Trọng số cuối cùng cho Monetary Value (M):", M_weight)


Trọng số cuối cùng cho Recency (R): 0.538961038961039
Trọng số cuối cùng cho Frequency (F): 0.2972582972582973
Trọng số cuối cùng cho Monetary Value (M): 0.16378066378066378


In [38]:
df_rfm 

,Recency,Frequency,MonetaryValue,Cluster
Customer ID,,,,
12347.0,3,71,1323.32,1
12348.0,74,20,222.16,1
12349.0,43,107,2646.99,1
12351.0,11,21,300.93,1
12352.0,11,18,343.80,1
...,...,...,...,...
18283.0,18,230,641.77,1
18284.0,65,29,436.68,1
18285.0,296,12,427.00,1


In [39]:
df_rfm[df_rfm['Cluster'] == 1]

,Recency,Frequency,MonetaryValue,Cluster
Customer ID,,,,
12347.0,3,71,1323.32,1
12348.0,74,20,222.16,1
12349.0,43,107,2646.99,1
12351.0,11,21,300.93,1
12352.0,11,18,343.80,1
...,...,...,...,...
18283.0,18,230,641.77,1
18284.0,65,29,436.68,1
18285.0,296,12,427.00,1


**Tính toán giá trị chuẩn hóa của R,F,M**

In [40]:
# Tính toán giá trị min và max của cột 'Recency'
min_recency = df_rfm['Recency'].min()
max_recency = df_rfm['Recency'].max()

# Tạo cột mới 'R_Norm' và tính toán giá trị cho từng hàng
df_rfm['R_Norm'] = df_rfm['Recency'].apply(lambda x: ((x - min_recency) / (max_recency - min_recency)) * (1 - 0) + 0)

# In ra DataFrame sau khi thêm cột mới
print(df_rfm)

             Recency  Frequency  MonetaryValue  Cluster    R_Norm
Customer ID                                                      
12347.0            3         71        1323.32        1  0.005362
12348.0           74         20         222.16        1  0.195710
12349.0           43        107        2646.99        1  0.112601
12351.0           11         21         300.93        1  0.026810
12352.0           11         18         343.80        1  0.026810
...              ...        ...            ...      ...       ...
18283.0           18        230         641.77        1  0.045576
18284.0           65         29         436.68        1  0.171582
18285.0          296         12         427.00        1  0.790885
18286.0          112         70        1188.43        1  0.297587
18287.0           18         86        2340.61        1  0.045576

[4291 rows x 5 columns]


In [41]:
# Tính toán giá trị min và max của cột 'Recency'
min_frequency = df_rfm['Frequency'].min()
max_frequency = df_rfm['Frequency'].max()

# Tạo cột mới 'R_Norm' và tính toán giá trị cho từng hàng
df_rfm['F_Norm'] = df_rfm['Frequency'].apply(lambda x: ((x - min_frequency) / (max_frequency - min_frequency)) * (1 - 0) + 0)

# In ra DataFrame sau khi thêm cột mới
print(df_rfm)

             Recency  Frequency  MonetaryValue  Cluster    R_Norm    F_Norm
Customer ID                                                                
12347.0            3         71        1323.32        1  0.005362  0.012261
12348.0           74         20         222.16        1  0.195710  0.003328
12349.0           43        107        2646.99        1  0.112601  0.018567
12351.0           11         21         300.93        1  0.026810  0.003503
12352.0           11         18         343.80        1  0.026810  0.002978
...              ...        ...            ...      ...       ...       ...
18283.0           18        230         641.77        1  0.045576  0.040112
18284.0           65         29         436.68        1  0.171582  0.004905
18285.0          296         12         427.00        1  0.790885  0.001927
18286.0          112         70        1188.43        1  0.297587  0.012086
18287.0           18         86        2340.61        1  0.045576  0.014889

[4291 rows 

In [42]:
# Tính toán giá trị min và max của cột 'Recency'
min_monetary = df_rfm['MonetaryValue'].min()
max_monetary = df_rfm['MonetaryValue'].max()

# Tạo cột mới 'R_Norm' và tính toán giá trị cho từng hàng
df_rfm['M_Norm'] = df_rfm['MonetaryValue'].apply(lambda x: ((x - min_monetary) / (max_monetary - min_monetary)) * (1 - 0) + 0)

# In ra DataFrame sau khi thêm cột mới
print(df_rfm)

             Recency  Frequency  MonetaryValue  Cluster    R_Norm    F_Norm  \
Customer ID                                                                   
12347.0            3         71        1323.32        1  0.005362  0.012261   
12348.0           74         20         222.16        1  0.195710  0.003328   
12349.0           43        107        2646.99        1  0.112601  0.018567   
12351.0           11         21         300.93        1  0.026810  0.003503   
12352.0           11         18         343.80        1  0.026810  0.002978   
...              ...        ...            ...      ...       ...       ...   
18283.0           18        230         641.77        1  0.045576  0.040112   
18284.0           65         29         436.68        1  0.171582  0.004905   
18285.0          296         12         427.00        1  0.790885  0.001927   
18286.0          112         70        1188.43        1  0.297587  0.012086   
18287.0           18         86        2340.61      

In [43]:
df_rfm

,Recency,Frequency,MonetaryValue,Cluster,R_Norm,F_Norm,M_Norm
Customer ID,,,,,,,
12347.0,3,71,1323.32,1,0.005362,0.012261,0.003872
12348.0,74,20,222.16,1,0.195710,0.003328,0.000650
12349.0,43,107,2646.99,1,0.112601,0.018567,0.007745
12351.0,11,21,300.93,1,0.026810,0.003503,0.000880
12352.0,11,18,343.80,1,0.026810,0.002978,0.001006
...,...,...,...,...,...,...,...
18283.0,18,230,641.77,1,0.045576,0.040112,0.001878
18284.0,65,29,436.68,1,0.171582,0.004905,0.001278
18285.0,296,12,427.00,1,0.790885,0.001927,0.001249


In [44]:
print("Trọng số cuối cùng cho Recency (R):", R_weight)
print("Trọng số cuối cùng cho Frequency (F):", F_weight)
print("Trọng số cuối cùng cho Monetary Value (M):", M_weight)

Trọng số cuối cùng cho Recency (R): 0.538961038961039
Trọng số cuối cùng cho Frequency (F): 0.2972582972582973
Trọng số cuối cùng cho Monetary Value (M): 0.16378066378066378


**Tính giá trị CLV cho từng khách hàng**

In [45]:
# Tính toán giá trị của cột 'CLV'
df_rfm['CLV'] = df_rfm['R_Norm'] * R_weight + df_rfm['F_Norm'] * F_weight + df_rfm['M_Norm'] * M_weight

# In ra DataFrame sau khi thêm cột mới
print(df_rfm)


             Recency  Frequency  MonetaryValue  Cluster    R_Norm    F_Norm  \
Customer ID                                                                   
12347.0            3         71        1323.32        1  0.005362  0.012261   
12348.0           74         20         222.16        1  0.195710  0.003328   
12349.0           43        107        2646.99        1  0.112601  0.018567   
12351.0           11         21         300.93        1  0.026810  0.003503   
12352.0           11         18         343.80        1  0.026810  0.002978   
...              ...        ...            ...      ...       ...       ...   
18283.0           18        230         641.77        1  0.045576  0.040112   
18284.0           65         29         436.68        1  0.171582  0.004905   
18285.0          296         12         427.00        1  0.790885  0.001927   
18286.0          112         70        1188.43        1  0.297587  0.012086   
18287.0           18         86        2340.61      

In [46]:
df_rfm

,Recency,Frequency,MonetaryValue,Cluster,R_Norm,F_Norm,M_Norm,CLV
Customer ID,,,,,,,,
12347.0,3,71,1323.32,1,0.005362,0.012261,0.003872,0.007169
12348.0,74,20,222.16,1,0.195710,0.003328,0.000650,0.106576
12349.0,43,107,2646.99,1,0.112601,0.018567,0.007745,0.067475
12351.0,11,21,300.93,1,0.026810,0.003503,0.000880,0.015635
12352.0,11,18,343.80,1,0.026810,0.002978,0.001006,0.015499
...,...,...,...,...,...,...,...,...
18283.0,18,230,641.77,1,0.045576,0.040112,0.001878,0.036795
18284.0,65,29,436.68,1,0.171582,0.004905,0.001278,0.094143
18285.0,296,12,427.00,1,0.790885,0.001927,0.001249,0.427033


**Tính toán giá trị CLV cho từng cụm khách hàng**

In [47]:
# Nhóm các dòng theo giá trị của cột 'Cluster' và tính giá trị trung bình cho mỗi nhóm
rfm_cluster = df_rfm.groupby('Cluster').mean().reset_index()

# In ra DataFrame mới
print(rfm_cluster)


   Cluster    Recency  Frequency  MonetaryValue    R_Norm    F_Norm    M_Norm  \
0        0   5.600000  2366.4000  206931.732000  0.012332  0.414328  0.605459   
1        1  88.703453    94.4937    1728.465549  0.235130  0.016377  0.005057   

        CLV  
0  0.228972  
1  0.132422  


In [48]:
rfm_cluster 

,Cluster,Recency,Frequency,MonetaryValue,R_Norm,F_Norm,M_Norm,CLV
0,0,5.600000,2366.4000,206931.732000,0.012332,0.414328,0.605459,0.228972
1,1,88.703453,94.4937,1728.465549,0.235130,0.016377,0.005057,0.132422
